In [1]:
import sys
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns 

windows_dir = "../kaggle/all/"
ubuntu_dir = "/home/prof/kaggle/unbankedcredit/kaggle/all/"

import platform

system = platform.system()

print("Notebook is running on operating system: {}".format(system))
if system == "Linux":
    directory = ubuntu_dir
else:
    directory = windows_dir


Notebook is running on operating system: Linux


In [2]:
# List files available

print(os.listdir(directory))


['sample_submission.csv', 'application_test.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'HomeCredit_columns_description.csv', 'bureau.csv', 'application_train.csv', 'credit_card_balance.csv', '.~lock.HomeCredit_columns_description.csv#', 'bureau_balance.csv']


Skipping bureau balance. It just contains dates relating to bureau balance but data doesn't include bureau balance. Sample_submission is an example of how to make the submission file

In [3]:
# # Bureau balance
# train_file = "{}bureau_balance.csv".format(directory)
# train = pd.read_csv(train_file)
# print('Training data shape: ', train.shape)
# train.head(30)

In [4]:
# # Sample Submission
# sub_file = "{}sample_submission.csv".format(directory)
# sub = pd.read_csv(sub_file)
# print('Training data shape: ', sub.shape)
# sub.head(30)

In [5]:
# # Installments 
install_file = "{}installments_payments.csv".format(directory)
installments = pd.read_csv(install_file, low_memory=False)
print('Training data shape: ', installments.shape)
installments.head(30)

Training data shape:  (13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
5,1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375
6,2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165
8,2723183,112102,0.0,14,-197.0,-197.0,70.740,70.740
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470


DAYS_INSTALMENT = number of days before loan payment was due
DAYS_ENTRY_PAYMENT = number of days before loan when payment occurred
so if DAYS_ENTRY_PAYMENT is smaller number than DAYS_INSTALLEMENT the payment was late
I think installment number is the number of times the person has repaid ie 7 = 7th repayment
AMT_INSTALMENT is the amount due and AMT_PAYMENT is actual payment so if AMT_PAYMENT is less then there is under payment

In [6]:
installments['DAYS_IN_ADVANCE'] = installments['DAYS_INSTALMENT'] - installments['DAYS_ENTRY_PAYMENT']

In [7]:
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_IN_ADVANCE
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,7.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,8.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-17.0


In [8]:
installments['AMT_UNDERPAYMENT'] = installments['AMT_INSTALMENT'] - installments['AMT_PAYMENT']

In [14]:
installments.to_csv('installments_payments_enhanced.csv', index=False)

In [10]:
installments[installments['DAYS_IN_ADVANCE'] <0]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_IN_ADVANCE,AMT_UNDERPAYMENT
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-17.0,4.455
19,2329072,183463,1.0,11,-1699.0,-1693.0,7283.295,7283.295,-6.0,0.000
30,1840786,175114,1.0,11,-476.0,-474.0,21179.700,21161.700,-2.0,18.000
45,1644499,111587,1.0,8,-2116.0,-2101.0,3078.810,3078.810,-15.0,0.000
47,1895324,131465,1.0,6,-1386.0,-1379.0,3229.065,3212.865,-7.0,16.200
48,2411245,153352,1.0,1,-2588.0,-2555.0,3294.900,650.520,-33.0,2644.380
68,1780053,189947,1.0,5,-374.0,-373.0,22772.340,22772.340,-1.0,0.000
69,1775456,134670,1.0,1,-981.0,-977.0,34703.820,32537.790,-4.0,2166.030
77,2436675,145728,1.0,15,-380.0,-378.0,13000.500,12382.560,-2.0,617.940
88,1915769,138380,1.0,17,-1067.0,-1060.0,16729.155,15592.365,-7.0,1136.790


In [11]:
installments[installments['AMT_UNDERPAYMENT'] >0]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_IN_ADVANCE,AMT_UNDERPAYMENT
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-17.0,4.455
17,1410565,197687,1.0,1,-1037.0,-1048.0,12514.050,12510.450,11.0,3.600
30,1840786,175114,1.0,11,-476.0,-474.0,21179.700,21161.700,-2.0,18.000
40,2270983,159586,0.0,30,-2022.0,-2046.0,9000.000,1800.000,24.0,7200.000
47,1895324,131465,1.0,6,-1386.0,-1379.0,3229.065,3212.865,-7.0,16.200
48,2411245,153352,1.0,1,-2588.0,-2555.0,3294.900,650.520,-33.0,2644.380
50,1220449,136423,1.0,16,-61.0,-89.0,34717.500,2675.745,28.0,32041.755
52,2726751,182114,1.0,20,-1066.0,-1091.0,10770.750,499.500,25.0,10271.250
61,1457055,109787,1.0,10,-1165.0,-1186.0,13016.115,13016.025,21.0,0.090
69,1775456,134670,1.0,1,-981.0,-977.0,34703.820,32537.790,-4.0,2166.030


In [12]:
# train enhanced
# train_file = "/home/prof/kaggle/unbankedcredit/train_enhanced_bur.csv"
# train = pd.read_csv(train_file, low_memory=False)
# print('Training data shape: ', train.shape)
# train.head()

Training data shape:  (307511, 271)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,DAYS_CREDIT_ENDDATE_mean,DAYS_ENDDATE_FACT_mean,AMT_CREDIT_MAX_OVERDUE_mean,CNT_CREDIT_PROLONG_mean,AMT_CREDIT_SUM_mean,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_OVERDUE_mean,DAYS_CREDIT_UPDATE_mean,AMT_ANNUITY_mean
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,-349.0,-697.500000,1681.029,0.0,108131.945625,49156.2,7997.14125,0.0,-499.875,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,-544.5,-1097.333333,0.000,0.0,254350.125000,0.0,202500.00000,0.0,-816.000,NaN
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,-488.5,-532.500000,0.000,0.0,94518.900000,0.0,0.00000,0.0,-532.000,NaN
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,-783.0,-783.000000,0.000,0.0,146250.000000,0.0,0.00000,0.0,-783.000,NaN


In [13]:
# train_enhanced_loan_history = pd.merge(train,installments, on='SK_ID_CURR', how='left')

MemoryError: 